In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sns.set_theme(style="whitegrid")

# Notes

If we use tags as predictors, we will end up with a lot of predictors.
Hence, we need some sort of variable selection method.
We can use lasso, stepwise, etc.

In [3]:
books = pd.read_csv("./goodbooks-10k/books.csv")
book_tags = pd.read_csv("./goodbooks-10k/book_tags.csv")
ratings = pd.read_csv("./goodbooks-10k/ratings.csv")
tags = pd.read_csv("./goodbooks-10k/tags.csv")
to_read = pd.read_csv("goodbooks-10k/to_read.csv")

In [4]:
print(len(ratings.user_id.unique()))

53424


In [5]:
ratings.describe()

,user_id,book_id,rating
count,5.976479e+06,5.976479e+06,5.976479e+06
mean,2.622446e+04,2.006477e+03,3.919866e+00
std,1.541323e+04,2.468499e+03,9.910868e-01
min,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.281300e+04,1.980000e+02,3.000000e+00
50%,2.593800e+04,8.850000e+02,4.000000e+00
75%,3.950900e+04,2.973000e+03,5.000000e+00
max,5.342400e+04,1.000000e+04,5.000000e+00


In [6]:
len(ratings.user_id.unique())

53424

In [7]:
books.describe()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn13,original_publication_year,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
count,10000.00000,1.000000e+04,1.000000e+04,1.000000e+04,10000.000000,9.415000e+03,9979.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04
mean,5000.50000,5.264697e+06,5.471214e+06,8.646183e+06,75.712700,9.755044e+12,1981.987674,4.002191,5.400124e+04,5.968732e+04,2919.955300,1345.040600,3110.885000,11475.893800,1.996570e+04,2.378981e+04
std,2886.89568,7.575462e+06,7.827330e+06,1.175106e+07,170.470728,4.428619e+11,152.576665,0.254427,1.573700e+05,1.678038e+05,6124.378132,6635.626263,9717.123578,28546.449183,5.144736e+04,7.976889e+04
min,1.00000,1.000000e+00,1.000000e+00,8.700000e+01,1.000000,1.951703e+08,-1750.000000,2.470000,2.716000e+03,5.510000e+03,3.000000,11.000000,30.000000,323.000000,7.500000e+02,7.540000e+02
25%,2500.75000,4.627575e+04,4.791175e+04,1.008841e+06,23.000000,9.780316e+12,1990.000000,3.850000,1.356875e+04,1.543875e+04,694.000000,196.000000,656.000000,3112.000000,5.405750e+03,5.334000e+03
50%,5000.50000,3.949655e+05,4.251235e+05,2.719524e+06,40.000000,9.780452e+12,2004.000000,4.020000,2.115550e+04,2.383250e+04,1402.000000,391.000000,1163.000000,4894.000000,8.269500e+03,8.836000e+03
75%,7500.25000,9.382225e+06,9.636112e+06,1.451775e+07,67.000000,9.780831e+12,2011.000000,4.180000,4.105350e+04,4.591500e+04,2744.250000,885.000000,2353.250000,9287.000000,1.602350e+04,1.730450e+04
max,10000.00000,3.328864e+07,3.553423e+07,5.639960e+07,3455.000000,9.790008e+12,2017.000000,4.820000,4.780653e+06,4.942365e+06,155254.000000,456191.000000,436802.000000,793319.000000,1.481305e+06,3.011543e+06


In [8]:
books.apply(lambda x: len(x.unique()), axis = 0)

book_id                      10000
goodreads_book_id            10000
best_book_id                 10000
work_id                      10000
books_count                    597
isbn                          9301
isbn13                        9154
authors                       4664
original_publication_year      294
original_title                9275
title                         9964
language_code                   26
average_rating                 184
ratings_count                 9003
work_ratings_count            9053
work_text_reviews_count       4581
ratings_1                     2630
ratings_2                     4117
ratings_3                     6972
ratings_4                     7762
ratings_5                     8103
image_url                     6669
small_image_url               6669
dtype: int64

In [9]:
ratings.shape

(5976479, 3)

In [10]:
tags.shape

(34252, 2)

In [11]:
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [12]:
(
    book_tags.merge(tags, on = "tag_id")
    .query("tag_id != 30574")
    .query("count > 5000")
    .sort_values("count", ascending=False)
)
     

,goodreads_book_id,tag_id,count,tag_name
30182,2548866,8717,77785,currently-reading
25759,24280,8717,64311,currently-reading
20359,2767052,11557,50755,favorites
14244,3,11557,48220,favorites
26313,38447,8717,47919,currently-reading
...,...,...,...,...
30974,7090447,8717,5013,currently-reading
38065,7126,11743,5012,fiction
12413,887877,11305,5011,fantasy
38866,18512,11743,5010,fiction


In [21]:
len(books.work_id.unique())

10000

In [23]:
len(books.goodreads_book_id.unique())

10000

## Tags

- There are 
- Some tags should be ignored
    - ex: currently-reading, to-read
- Some tags are equivalent but spelled differently
    - ex: favorites, favourites, F a v o r i t e s

- Choose p tags as predictors

In [13]:
TAGS_TO_IGNORE = {30574: "to-read", 
                  8717: "currently-reading",
                 }

popular_tags = (
    book_tags.merge(tags, on = "tag_id")
    .query("tag_id not in @TAGS_TO_IGNORE.keys()")
    .query("count > 2000")
    .tag_name
    .unique()
)

# Top raters

In [14]:
ratings

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


In [15]:
(
    ratings
    .groupby("user_id", as_index=False)
    .count()
    .sort_values("rating", ascending=False)
)

,user_id,book_id,rating
30943,30944,200,200
12873,12874,200,200
52035,52036,199,199
12380,12381,199,199
28157,28158,199,199
...,...,...,...
32127,32128,21,21
40752,40753,21,21
51724,51725,21,21
43674,43675,20,20


In [16]:
top_user = 30944

## Tags per book

In [17]:
(
    book_tags
    .merge(tags, on="tag_id")
    .groupby(["goodreads_book_id"], as_index=False)
    .count()
    .sort_values("count", ascending=False)
)

,goodreads_book_id,tag_id,count,tag_name
0,1,100,100,100
6668,6344097,100,100,100
6661,6338619,100,100,100
6662,6339304,100,100,100
6663,6339664,100,100,100
...,...,...,...,...
3335,91571,100,100,100
9999,33288638,100,100,100
4650,280958,94,94,94
9220,18607805,62,62,62


# Top tags

In [18]:
top_tags = (
    book_tags
    .merge(books, on="goodreads_book_id")
    .filter(items=["count", "goodreads_book_id", "tag_id", "authors", "ratings_count", "original_title"])
    .merge(tags, on="tag_id")
    .groupby("tag_name", as_index=False)
    .sum()
    .sort_values("count", ascending=False)
    #.query("tag_name != 'to-read'")
    .head(10)
)

In [19]:
top_tags

,tag_name,count,goodreads_book_id,tag_id,ratings_count
30574,to-read,140718761,52428278826,305220242,539430823
8717,currently-reading,7507958,50864864557,85217392,514150222
11557,favorites,4503173,52111252697,114194717,524652206
11743,fiction,3688819,48136824753,106826071,512622984
11305,fantasy,3548157,19754086947,48147995,273906403
33114,young-adult,1848306,21650946753,120203820,290167626
7457,classics,1756920,1611584464,20767745,229013859
5207,books-i-own,1317235,51362896962,51023393,519363456
26138,romance,1231926,28792484719,111112638,248335972
22743,owned,1224279,51887174731,224200494,527289733
